In [ ]:
import time
import requests
import datetime
import numpy as np
import pandas as pd

from itertools import groupby
from tableau_publish import Tableau

class Json2Tableau:
    def __init__(self) -> None:
        self.log_path = 'U:/weather2tableau/weatherLog.txt'

        self.authorization = "rdec-key-123-45678-011121314"

        self.url = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-D0047-055?Authorization=" + self.authorization + "&format=JSON"
    
    # for 上拋tableau 不用更動
    def typeList(self, df):        
        _list = []
        for i in range(len(df.columns)):
            _type = str(type(df.iloc[0, i]))
            #print(_type)
            if 'Timestamp' in _type:
                _list.append('datetime')
            elif 'int64' in _type:
                _list.append('int')
            elif 'float' in _type:
                _list.append('float')
            elif 'bool' in _type:
                _list.append('bool')
            elif 'str' in _type:
                _list.append('str')
            else:
                _list.append('str')
        print(_list)
        return _list

    def get_weather_weekly_forecast(self, path):
        '''
        https://data.gov.tw/dataset/9308
        '''
        # Taiwan
        #url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-D0047-091?Authorization=CWB-9A174D42-4149-44D7-83C0-CDEE16C38497'
        # HC
        # url = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-D0047-055?Authorization=CWB-9A174D42-4149-44D7-83C0-CDEE16C38497&format=JSON"
        
        flag = 1
        
        while flag:
            try:
                http_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
                https_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
                ftp_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
                
                proxyDict = { 
                            "http"  : http_proxy, 
                            "https" : https_proxy, 
                            "ftp"   : ftp_proxy
                            }
                
                r = requests.get(self.url, proxies=proxyDict, verify=False)
                
                # Parse
                data = pd.read_json(r.text)
                data = data.loc['locations', 'records']
                data = data[0]['location']
            
                # Fetch Data ......
                results = pd.DataFrame()
            
                # 該縣市所有地區
                for i in range(len(data)):
                    loc_data = data[i]
                    
                    loc_name = loc_data['locationName']
                    geocode = loc_data['geocode']
                    lat = loc_data['lat']
                    lon = loc_data['lon']
                    weather_data = loc_data['weatherElement']
            
                    # 資料類型，有15個
                    # 0              PoP12h 12小時降雨機率
                    # 1                   T 平均溫度
                    # 2                  RH 平均相對濕度
                    # 3               MinCI 最小舒適度指數
                    # 4                  WS 最大風速
                    # 5               MaxAT 最高體感溫度
                    # 6                  Wx 天氣現象
                    # 7               MaxCI 最大舒適度指數
                    # 8                MinT 最低溫度
                    # 9                UVI 紫外獻指數
                    # 10 WeatherDescription 天氣預報綜合描述
                    # 11              MinAT 最低體感溫度
                    # 12               MaxT 最高溫度
                    # 13                 WD 風向
                    # 14                 Td 平均露點溫度
                    
                    
                    for j in range(len((weather_data))):
                        ele_data_dict = weather_data[j]
                        
                        for k in range(len(ele_data_dict)):
                            ele_name = ele_data_dict['elementName']
                            ele_desc = ele_data_dict['description']
                            ele_data = ele_data_dict['time']
            
                            # 此欄位為質性資料，如「'陰短暫雨。降雨機率 90%。溫度攝氏18至22度。
                            # 舒適。東北風 風速5級(每秒8公尺)。相對濕度92%。'」，因此不保留
                            if ele_name == 'WeatherDescription':
                                continue
                            
                            for l in range(len(ele_data)):
                                start_time = ele_data[l]['startTime']
                                end_time = ele_data[l]['endTime']
                                value = ele_data[l]['elementValue'][0]['value']
                                
                                # 先保留全部的資料，最後再決定要保留哪些欄位
                                new_data = \
                                    pd.DataFrame({'location':[loc_name],
                                                'geocode':[geocode],
                                                'lat':[lat],
                                                'lon':[lon],
                                                'element':[ele_name],
                                                'description':[ele_desc],
                                                'start_time':[start_time],
                                                'end_time':[end_time],
                                                'value':[value]})
                                
                                results = results.append(new_data)
                    print('update_weekly_forecast' + str(i) + '/' + str(len(data)))
                
                results = results.reset_index(drop=True)
                results.to_csv(path + '/weather.csv', index=False, encoding='utf-8-sig')

                # 結束迴圈
                flag = 0
                return results
            except:
                print("Connection refused by the server..")
                print("Let me sleep for 5 seconds")
                print("ZZzzzz...")
                time.sleep(5)
                print("Was a nice sleep, now let me continue...")
                continue

if __name__=='__main__':
    Json2Tableau = Json2Tableau()

    start = time.time()
    tbl_df = Json2Tableau.get_weather_weekly_forecast("U:/weather2tableau")

    # 上拋至Tableau
    try:
        tbl = Tableau()
        tbl.__basic__([tbl_df.columns.values.tolist()] + tbl_df.values.tolist(), 
                    Json2Tableau.typeList(tbl_df),
                    'LCD1', # 頂層資料夾名稱
                    'LCD1_weather', # 資料夾名稱
                    'gcp-edatbl07',
                    'account', # 輸入帳號
                    'password', # 輸入密碼
                    'FE')
        tbl.csv2hyper()
        
        end = time.time()
        with open(Json2Tableau.log_path, 'a') as f:
            f.write(str(datetime.datetime.now()) + ', Upload to tableau dataset success take time: ' + str(end-start)+'\n')
    except:
        with open(Json2Tableau.log_path, 'a') as f:
            f.write(str(datetime.datetime.now()) + ', Upload to tableau dataset failed!\n')

    print ("Time: ", end-start)